In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df_attri = pd.read_csv('attribution_allocation_student_data.csv')
df_chan = pd.read_csv('channel_spend_student_data.csv')

In [3]:
df_attri.describe()

,tier
count,68124.000000
mean,2.333333
std,0.745361
min,1.000000
25%,2.000000
50%,2.500000
75%,3.000000
max,3.000000


In [4]:
df_attri.head()

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1


In [5]:
type(df_chan['0'][0])

str

In [6]:
s = {}
for i in range(len(df_chan)):
    s["spend_{}".format(i + 1)] = eval(df_chan['0'][i])
    #s["spend_{}".format(i + 1)] = pd.DataFrame.from_dict(dic, orient='index', columns=['Spend'])

In [7]:
s["spend_1"]

{'email': 1000.0,
 'social': 1000.0000000000001,
 'display': 1000.0,
 'paid_search': 999.9999999999999,
 'referral': 1000.0,
 'organic_search': 0.0,
 'direct': 0.0}

# First Interaction Model

In [10]:
df_attri_T = df_attri[df_attri['convert_TF']== True]

In [11]:
df_attri_T = df_attri_T.reset_index()

In [12]:
df_attri_T.describe()

,index,tier
count,13779.000000,13779.000000
mean,26608.898541,2.235068
std,18268.800880,0.767954
min,0.000000,1.000000
25%,12388.500000,2.000000
50%,26056.000000,2.000000
75%,38164.500000,3.000000
max,68117.000000,3.000000


In [13]:
result1 = df_attri_T['touch1'].value_counts().rename_axis('channel')
result1 = pd.DataFrame(result1)
result1

,touch1
channel,
referral,7495
social,2280
display,2031
email,1284
paid_search,668
organic_search,15
direct,6


In [14]:
type(result1)

pandas.core.frame.DataFrame

In [15]:
#df_attri_T.groupby(['tier','touch1']).count()

# CAC for each channel

In [16]:
CAC_FI={}
for i in result1.index:
    CAC_FI["CAC_{}".format(i)]=float((s["spend_1"][i]+s["spend_2"][i]+s["spend_3"][i])
                                  /result1.loc[i,'touch1'])
CAC_FI

{'CAC_referral': 0.800533689126084,
 'CAC_social': 2.6315789473684212,
 'CAC_display': 2.9542097488921715,
 'CAC_email': 4.672897196261682,
 'CAC_paid_search': 8.982035928143713,
 'CAC_organic_search': 0.0,
 'CAC_direct': 0.0}

# Linear Model

In [17]:
#df_attri_T.groupby()

In [18]:
#temp = df_attri_T['touch1'].value_counts()
#temp = pd.DataFrame(temp)

In [19]:
#temp

In [20]:
#temp2 = df_attri_T['touch2'].value_counts()
#temp2 = pd.DataFrame(temp2)

In [21]:
#temp2

In [22]:
#for i in np.arange(2,6):
#    temp1 = df_attri_T.iloc[:,i].value_counts()
#    temp1 = pd.DataFrame(temp1)
#    temp = temp.join(temp1)
#print(temp)

In [23]:
#type(temp)

In [24]:
num_null = df_attri_T.isnull().sum(axis=1)

In [25]:
linear = pd.DataFrame({'referral':[0],'social':[0], 'display':[0], 
                       'email':[0], 'paid_search':[0],'organic_search':[0],'direct':[0]})
linear

,referral,social,display,email,paid_search,organic_search,direct
0,0,0,0,0,0,0,0


In [26]:
#df_attri_T.head(10)

In [27]:
for i in np.arange(len(df_attri_T)):
    j=1
    credit = 1/(5-num_null[i])
    while j<(6-num_null[i]):
        linear.loc[:,df_attri_T.iloc[i,1+j]]+= credit
        j += 1    

In [28]:
#df_attri_T.iloc[:,7]

In [29]:
linear.sum().sum()

13778.999999998588

In [30]:
type(linear)

pandas.core.frame.DataFrame

In [31]:
len(df_attri_T)

13779

In [32]:
linear

,referral,social,display,email,paid_search,organic_search,direct
0,7463.833333,2302.183333,2009.466667,1275.1,704.866667,16.15,7.4


# CAC for each channel

In [33]:
CAC_linear={}
list_linear = list(linear.columns.values)
for i in np.arange(len(list_linear)):
    CAC_linear["CAC_{}".format(list_linear[i])]=float((s["spend_1"][list_linear[i]]+s["spend_2"][list_linear[i]]+s["spend_3"][list_linear[i]])
                                  /linear.loc[0,list_linear[i]])
CAC_linear

{'CAC_referral': 0.8038764709824394,
 'CAC_social': 2.6062216301916488,
 'CAC_display': 2.9858668966890645,
 'CAC_email': 4.705513293075028,
 'CAC_paid_search': 8.512248179324631,
 'CAC_organic_search': 0.0,
 'CAC_direct': 0.0}

# Last interaction Model

In [34]:
out1 = df_attri_T.replace('', np.NaN).ffill(axis=1).iloc[:, -2]
print(out1)

0              email
1              email
2           referral
3              email
4           referral
            ...     
13774       referral
13775       referral
13776    paid_search
13777        display
13778         direct
Name: touch5, Length: 13779, dtype: object


In [35]:
out1.value_counts()

referral          6014
social            2960
display           2292
email             1491
paid_search       1004
organic_search      11
direct               7
Name: touch5, dtype: int64

In [36]:
result3 = out1.value_counts().rename_axis('channel')
result3 = pd.DataFrame(result3)
result3

,touch5
channel,
referral,6014
social,2960
display,2292
email,1491
paid_search,1004
organic_search,11
direct,7


# CAC for each channel

In [37]:
CAC_LI={}
for i in result3.index:
    CAC_LI["CAC_{}".format(i)]=float((s["spend_1"][i]+s["spend_2"][i]+s["spend_3"][i])
                                  /result3.loc[i,'touch5'])
CAC_LI

{'CAC_referral': 0.9976720984369803,
 'CAC_social': 2.027027027027027,
 'CAC_display': 2.6178010471204187,
 'CAC_email': 4.0241448692152915,
 'CAC_paid_search': 5.9760956175298805,
 'CAC_organic_search': 0.0,
 'CAC_direct': 0.0}